In [3]:
import cv2
from scenedetect import VideoManager, SceneManager, FrameTimecode, StatsManager
from scenedetect.detectors import ContentDetector

# 동영상 열기
cap = cv2.VideoCapture('/workspace/scene_change/IMG_4247.mp4')

# 프레임 추출
frame_count = 0
prev_frame = None
scene_list = []  # scene_list 변수 정의

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 프레임 그레이스케일 변환
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 이전 프레임과 현재 프레임의 차이 계산
    if prev_frame is not None:
        diff = cv2.absdiff(gray, prev_frame)
        count = cv2.countNonZero(diff)

        # 임계값 이상의 차이가 감지되면 해당 시간 프레임 저장
        if count > 5000:
            time_ms = int(cap.get(cv2.CAP_PROP_POS_MSEC))
            timecode = FrameTimecode(time_ms, fps=cap.get(cv2.CAP_PROP_FPS))
            scene_list.append(timecode)

    prev_frame = gray.copy()
    frame_count += 1

# 동영상 종료 및 OpenCV 리소스 해제
cap.release()
cv2.destroyAllWindows()

# PySceneDetect를 사용하여 화면 장면 전환 감지
video_manager = VideoManager(['/workspace/scene_change/IMG_4247.mp4'])
stats_manager = StatsManager()
scene_manager = SceneManager(stats_manager)
scene_manager.add_detector(ContentDetector(threshold=30))
scene_manager.detect_scenes(frame_source=video_manager)

# 감지된 시간 프레임 출력
scene_list = scene_manager.get_scene_list()
for i, scene in enumerate(scene_list):
    print("Scene %2d: %s - %s" % (i+1, scene[0], scene[1]))

VideoManager is deprecated and will be removed.


In [4]:
import cv2
from scenedetect import SceneManager, StatsManager
from scenedetect.frame_timecode import FrameTimecode
from scenedetect.detectors import ContentDetector

# 동영상 열기
cap = cv2.VideoCapture('/workspace/scene_change/IMG_4247.mp4')

# PySceneDetect 객체 생성
video_fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
stats_manager = StatsManager()
scene_manager = SceneManager(stats_manager)
scene_manager.add_detector(ContentDetector(threshold=30))

# 화면 장면 전환 감지
scene_list = []
start_time = None
while True:
    # 프레임 읽기
    ret, frame = cap.read()
    if not ret:
        break

    # 현재 프레임의 시간 추출
    frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
    time_ms = int(cap.get(cv2.CAP_PROP_POS_MSEC))
    timecode = FrameTimecode(time_ms, fps=video_fps)

    # PySceneDetect에서 화면 장면 전환 감지
    scene_manager.detect_scenes(frame_number, frame)

    # 새로운 화면 장면이 감지되면 시작 시간 저장
    if scene_manager.get_num_scenes() > len(scene_list):
        start_time = timecode

    # 화면 장면이 종료되면 해당 구간의 시작 시간과 종료 시간 저장
    for i, scene in enumerate(scene_manager.get_scene_list()):
        if i > len(scene_list):
            start_frame, end_frame = scene[0], scene[1]
            end_time = timecode
            scene_list.append([start_time, end_time])
            start_time = None
            break

# 동영상 종료 및 OpenCV 리소스 해제
cap.release()
cv2.destroyAllWindows()

# 감지된 시간 프레임 출력
for i, scene in enumerate(scene_list):
    print("Scene %2d: %s - %s" % (i+1, scene[0], scene[1]))


ValueError: Timecode frame number must be positive and greater than zero.